In [1]:
import openpyxl
import pandas as pd
import os, docx2txt
import re

total_wi = pd.read_excel("Work_plan_3gpp_230110.xlsm")
wb = openpyxl.load_workbook('Work_plan_3gpp_230110.xlsm')
ws = wb['Work_plan_3gpp']
# date: 2023/2/19

# 下載work plan中會議記錄

## 1. 探索Work_plan_3gpp_230110.xlsm檔案

In [4]:
content = {}
valid_lines = 0

for i in range(1, len(ws['K'])+1):
    if ws[f'B{i}'].value != 0: 
        valid_lines += 1
        if ws.cell(i, 11).hyperlink:
            content[ws[f'K{i}'].value] = {
                        'Unique_ID': ws[f'B{i}'].value,
                        'release': ws[f'F{i}'].value,
                        'Hyperlink': ws.cell(i, 11).hyperlink.target,
                        'WI rapporteur name': ws[f'K{i}'].value,
                        'Supporting Individual Members':[] 
                    }

print(f'共{valid_lines}筆有效資料，部分release 11及其後會議附上會議紀錄，共{len(content)}筆')

共8144筆有效資料，部分release 11及其後會議附上會議紀錄，共2813筆


## 2. 下載excel中的所有檔案並解壓縮

In [5]:
import requests, zipfile, io


########## zipfile 參考用法 ##########

# r = requests.get(zip_file_url)
# z = zipfile.ZipFile(io.BytesIO(r.content))
# z.extractall("./doc")
######################################


suc_count = 0
fail_count = 0


for key, value in content.items():
    if key not in ' '.join(os.listdir('./docx')):
        try:
            r = requests.get(value['Hyperlink'])
            z = zipfile.ZipFile(io.BytesIO(r.content))
            z.extractall("./doc")
#             print(f'done: {key}',": ",content[key]['Hyperlink'], f"in {content[key]['release']}")
            suc_count += 1

        except:
#             print(f'Failed download: {key}',": ",content[key]['Hyperlink'], f"in {content[key]['release']}")
#             print(f"{key} in {content[key]['release']}")
            fail_count += 1 

    
print(f'finish download {suc_count} documents, with {fail_count} failed')

## 3. 使用excel VBA將doc轉換成docx
https://zh-tw.extendoffice.com/documents/word/5601-word-batch-convert-doc-to-docx.html

## 3. 讀取所有整理好的docx檔

In [3]:
df = pd.DataFrame(
    {'title':[],
     'name':[],
     'rapporteur':[],
     'supporting member':[],
     'file':[]
       })

In [14]:
import docx2txt
finish_crawler = 0
invalid_files = []
warn = {}

# for file in ['./CP-140450.docx']:

for file in os.listdir('./docx'):
    my_text = docx2txt.process(f"./docx/{file}").lower()
    file_name = file
    
    ########## 跳過: 正在使用的docx檔、非會議記錄的檔案 ##########
    if '~' in file_name or 'supporting individual members' not in my_text or 'rapporteur' not in my_text:
        invalid_files.append(file_name)
        print(file_name, 'seems not a conference file')
        
    else:    
        
        
        ########## 爬取會議名稱：通常在會議記錄右上角，否則也是會議記錄檔名 ##########
        try:
            name = [re.sub('^[^a-zA-Z]+',"", i) for i in my_text.split('\t') if re.match('[a-z][a-z]-', re.sub('^[^a-zA-Z]+',"", i))][0].split('\n')[0].replace(" ", "")[:9].upper()
            print(file_name)
            print(name)
        except:
            if re.match('[a-zA-Z][a-zA-Z]-', file_name):
                name = file_name[:9]
                print(file_name)
                print(name)
            else:
                name = ''
                print(file, f"can't find name")
                warn[file] = my_text.split('\t')

#         print(name)
#         if name.upper() not in content:
#             print(name.upper())


        ########## 爬取會議討論之Work Item標題：通常在title欄位 ##########
    
        try:    
            title = my_text.split('title')[1].split('\n')[0]
        except:    
            title = ''
            print(file, f"can't find title")
            
            
        ########## 爬取WI rapporteur：通常會在work item rapporteur(s)欄位 ##########
        
        if 'work item rapporteur(s)' in my_text:
#             WI_rapporteur = '; '.join([i for i in my_text.split('work item rapporteur(s)')[1].split('work item leadership')[0].split('\n\n') if len(i) > 2 and len(i) < 100 and 'rapporteur' not in i])
            WI_rapporteur = '; '.join([i for i in my_text.split('work item rapporteur(s)')[1].split('work item leadership')[0].split('\n\n') if len(i) > 3])
#             print('work item rapporteur(s)')
            
        elif 'study item rapporteur(s)' in my_text:
            WI_rapporteur = '; '.join([i for i in my_text.split('study item rapporteur(s)')[1].split('study item leadership')[0].split('\n\n') if len(i) > 3])
#             print('work item rapporteur(s)')
            
        elif 'study item rapporteur' in my_text:
            WI_rapporteur = '; '.join([i for i in my_text.split('study item rapporteur')[1].split('study item leadership')[0].split('\n\n') if len(i) > 3])
#             print('work item rapporteur(s)')
            
        else:
            try:    
                WI_rapporteur = '; '.join([i for i in my_text.split('work item rapporteur')[1].split('work item leadership')[0].split('\n\n') if len(i) > 3])
#                 print('work item rapporteur')
            except:    
                WI_rapporteur = ''
                print('****************',file, f"can't find WI_rapporteur")
                
                
        ########## 爬取WI supporting members：通常會在supporting individual members欄位 ##########

        if 'im name' in my_text:  
            supporting_ims = my_text.split('im name')[1].split('version')[0].split('\n\n')[1:]
        else:
            try:
                supporting_ims = my_text.split('supporting individual members')[1].split('\n\n')[0].replace('\t', '').replace('\n', '').split('\n\n')[1:]
            except:   
                supporting_ims = ''
                print(file, f"can't find supporting_ims")

#     print(supporting_ims)
#         if len(WI_rapporteur) > 100 or len(WI_rapporteur) < 3:
#             print(file)
#             print(re.sub('\s+'," ",re.sub('^\W+|[1-9\s;]+$',"",WI_rapporteur)))


        ########## 確認supporting members資料，每一個supporting member建一筆資料 ##########
    
        for im in supporting_ims:   
            im = re.sub('^[^a-zA-Z]+',"",im)
            if len(im) > 1 and len(im) < 31 and '.' not in im and not 'references' in im and not 'history' in im:
#                 print(im, 'len: ', len(im))
                
                dict = {'title': re.sub('^\W+',"",title), 
                        'name': name.upper(), 
                        'rapporteur': re.sub('\s+'," ",re.sub('^\W+|[1-9\s;]+$',"",WI_rapporteur)), 
                        'supporting member': im, 
                        'file': file_name}
                df2 = pd.DataFrame(dict, index=[0])
                df = pd.concat([df, df2], ignore_index = True)
    
    finish_crawler +=1
    if finish_crawler % 1000 == 0:
        print(f'=================finish: {finish_crawler}===================')

df.reset_index()

./CP-140450.docx
CP-140500



In [100]:
print('finish craw', finish_crawler, 'files')

print('==== invalid files: ', len(invalid_files), ' ====')
print(invalid_files)

finish craw 812 files
==== invalid files:  38  ====
['33203_CR0235R2_(Rel-13)_S3-152117.docx', '33203_CR0236R2_(Rel-13)_S3-152118.docx', '33203_CR0238_(Rel-13)_S3-151787.docx', '33203_CR0239_(Rel-13)_S3-151808 CR 33.203 WebRTC Scenario 4 v1.docx', 'C1-221266.docx', 'Cover.docx', 'CP-140405.docx', 'R4-160004.docx', 'RP-120826_LTE_CA_C_B1_Cover.docx', 'RP-130170_revisedWID_LTE_CA_B23_B29_Cover.docx', 'RP-130175 - Revised WID - LTE_CA_B2_B4-Cover.docx', 'RP-130179 - Updated WID LTE Advanced intra-band non-contiguous Carrier Aggregation in Band 25 (Cover) - clean.docx', 'RP-130179 - Updated WID LTE Advanced intra-band non-contiguous Carrier Aggregation in Band 25 (Cover).docx', 'RP-130181 Cover.docx', 'RP-130189_Revised_WID_CA_B3_B28(Cover).docx', 'RP-130197(Cover).docx', 'RP-130198(Cover).docx', 'RP-130396 - SID RAN enhancements for MTC.docx', 'RP-130668(Cover).docx', 'RP-130669(Cover).docx', 'RP-130813 Revised WID LTE_CA_NC_B7 (Cover).docx', 'RP-131135 - Revised WID - LTE_CA_NC_B4-Cover_

In [145]:
# df.to_excel('result.xlsx', encoding='utf_8_sig', index=False)

## 4. 補上WI ID，並檢視爬取失敗的檔案

In [5]:
# df = pd.read_excel("final_result.xlsx")
print('總資料筆數:', len(df))

總資料筆數: 45753


In [19]:
import time
invalid_name = []
unique_ID = []

for i in df.name:
    try:
        unique_ID.append(content[i]['Unique_ID'])
    except KeyError:
        time.sleep(0.2)
        unique_ID.append(' ')
        if df['file'].loc[df.name == i].iloc[0] not in invalid_name:
            invalid_name.append(df['file'].loc[df.name == i].iloc[0])

print(len(unique_ID))
print(invalid_name)

45753
['CP-140271.docx', 'CP-140450.docx', 'CP-183246_was CP-183129_New WID on enhancement of PCC related services.docx', 'RP-190213 revised WID_NR_clean.docx', 'RP-130416 WID - Support for BeiDou Navigation Satellite System (BDS) - Feature Part.docx', 'RP-130669_WID_CA_B1_B18_BWset(Performance).docx', 'RP-130221 Revised SID for Scalable UMTS FDD Bandwidth with trace.docx', 'RP-141037_AWS_EXT_v7.docx', 'RP-142202 B2_B4_B5_B30.docx', 'RP-160678 WID measurement gap enhancement_clean.docx', 'RP-160912.docx', 'RP-160954_clean.docx', 'RP-161291.docx', 'RP-161929_eLWIP-WID RAN3 v15.docx', 'RP-160210 above 6GHz_CM_revSID.docx', 'RP-162007 3.5GHz WID.docx', 'RP-172035 Revised TRP TRS WID RAN76_v1.docx', 'RP-171757 HPUE for B3 B20 B28 WID modified.docx', 'RP-193266 WI update no change marks.docx', 'RP-201253 Revised WID for NR CA_DC 2 band DL with up to 2 band UL_clean.docx', 'RP-213669 rev 3328 r2297_EPC UPIP WID_cln.docx', 'RP-223553.docx', 'RP-223529.docx', 'RP-223530.docx', 'RP-221481 Revis

In [23]:
df.insert(0, 'unique_ID', unique_ID)
df.to_excel('result_with_id.xlsx', encoding='utf_8_sig', index=False)